In [5]:
import tensorflow
import tensorflow as tf
import pandas as pd 

In [8]:
priv_dat = pd.read_excel("privateProdAllYears.xls", one_hot=True)
#priv_dat = pd.read_excel("privateProdAllYears.xls")
public_dat = pd.read_excel("publicProdAllYears.xls")

In [39]:
data= tf.transpose(priv_dat["Average"])
#print(data[4:10])
#print((data))


In [28]:
LR = 0.5
neurons = 10

In [30]:
a_0 = tf.placeholder(tf.float32, [None, 20])
y = tf.placeholder(tf.float32, [None, 1])

w_1 = tf.Variable(tf.truncated_normal([20, neurons]))
#b_1 = tf.Variable(tf.truncated_normal([1, middle]))
w_2 = tf.Variable(tf.truncated_normal([neurons, 1])) 
#b_2 = tf.Variable(tf.truncated_normal([1, 10]))

In [31]:
def sigma(x):
    return tf.div(tf.constant(1.0),
                  tf.add(tf.constant(1.0), tf.exp(tf.negative(x))))

def sigmaprime(x):
    return tf.multiply(sigma(x), tf.subtract(tf.constant(1.0), sigma(x)))


In [32]:
z_1 = tf.matmul(a_0, w_1)
a_1 = sigma(z_1)
z_2 = tf.matmul(a_1, w_2)
a_2 = sigma(z_2)

In [33]:
diff = tf.subtract(a_2, y)

In [34]:
d_z_2 = tf.multiply(diff, sigmaprime(z_2))
d_b_2 = d_z_2
d_w_2 = tf.matmul(tf.transpose(a_1), d_z_2)

d_a_1 = tf.matmul(d_z_2, tf.transpose(w_2))
d_z_1 = tf.multiply(d_a_1, sigmaprime(z_1))
d_b_1 = d_z_1
d_w_1 = tf.matmul(tf.transpose(a_0), d_z_1)

In [35]:
eta = tf.constant(LR)
cost = tf.multiply(diff, diff)
step = tf.train.GradientDescentOptimizer(LR).minimize(cost)
print(step)
#step = [
#    tf.assign(w_1,
#            tf.subtract(w_1, tf.multiply(eta, d_w_1)))
#  , tf.assign(w_2,
#            tf.subtract(w_2, tf.multiply(eta, d_w_2)))
#]

name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Variable/ApplyGradientDescent"
input: "^GradientDescent/update_Variable_1/ApplyGradientDescent"



In [4]:
acct_mat = tf.equal(tf.argmax(a_2, 1), tf.argmax(y, 1))
acct_res = tf.reduce_sum(tf.cast(acct_mat, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(10000):
    batch_xs, batch_ys = priv_dat.train.next_batch(10)
    sess.run(step, feed_dict = {a_0: batch_xs,
                                y : batch_ys})
    if i % 1000 == 0:
        res = sess.run(acct_res, feed_dict =
                       {a_0: priv_dat.test.images[:1000],
                        y : priv_dat.test.labels[:1000]})
        print (res)

NameError: name 'a_2' is not defined